<a href="https://colab.research.google.com/github/nickwan/nwds-stream-notebooks/blob/main/subtraction_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# font

In [ ]:
def increase_font():
  from IPython.display import Javascript
  display(Javascript('''
  for (rule of document.styleSheets[0].cssRules){
    if (rule.selectorText=='body') {
      rule.style.fontSize = '24px'
      break
    }
  }
  '''))
get_ipython().events.register('pre_run_cell', increase_font)

# subtraction method and stuff+

In [ ]:
!pip install pybaseball
!pip install catboost

<IPython.core.display.Javascript object>

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.1/426.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.0 MB/s eta 0:00:00


In [ ]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import seaborn as sns
import math
import pybaseball as bb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import KFold
import catboost as cb
from sklearn.metrics import mean_squared_error, log_loss
bb.cache.enable()

<IPython.core.display.Javascript object>

In [ ]:
project_dir = 'drive/My Drive/baseball/data'

<IPython.core.display.Javascript object>

In [ ]:
sc = pd.read_csv(f"{project_dir}/2025-data.csv")

# adjust for lefties
for col in ['pfx_x', 'release_pos_x']:
  sc[f"{col}_adj"] = sc[col].copy()
  sc.loc[sc['p_throws']=='L', f"{col}_adj"] = sc.loc[sc['p_throws']=='L', col].mul(-1)

# clean spin axis a little for the model
sc['spin_axis_adj'] = sc['spin_axis'].copy()
sc.loc[sc['p_throws']=='L', 'spin_axis_adj'] = sc.loc[sc['p_throws']=='L', 'spin_axis'].mul(-1).add(360)
sc.loc[:,'spin_axis_adj'] = sc.loc[:,'spin_axis_adj'].replace(360, 0)

# get movement avgs for delta
_df = sc.loc[sc['pitch_type'].isin(['FF','SI']), ['pitcher','release_speed','pfx_x_adj','pfx_z']].groupby(['pitcher'],as_index=False).mean().rename(columns={'release_speed':'release_speed_avg','pfx_x_adj':'pfx_x_adj_avg','pfx_z':'pfx_z_avg'})
sc = sc.merge(_df)

for col in ['release_speed', 'pfx_x_adj', 'pfx_z']:
  sc[f'{col}_diff'] = sc[col].sub(sc[f"{col}_avg"])

sc.shape

<IPython.core.display.Javascript object>

(721665, 131)

In [ ]:
sc.columns.values

<IPython.core.display.Javascript object>

array(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'pitcher_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year',
       'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'umpire', 'sv_id', 'vx0',
       'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle',
       'effective_speed', 'release_spin_rate', 'release_extension',
       'game_pk', 'fielder_2', 'fielder_3', 'fielder_4', 'fielder_5',
       'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9',
       'release_pos_y', 'estimated_ba_using_speedangle',
 

In [ ]:
feats = [
    'release_speed_diff', 'pfx_x_adj_diff', 'pfx_z_diff',
    'release_speed', 'pfx_x', 'pfx_z','release_spin_rate',
    'spin_axis_adj', 'release_pos_x_adj','release_extension',
    'release_pos_z'
]

target = 'delta_run_exp'


<IPython.core.display.Javascript object>

In [ ]:
model_data = sc.dropna(subset=feats+[target])
folds = 3
kf = KFold(n_splits=folds, shuffle=True)
s = pd.Series()
for train_idx, test_idx in kf.split(model_data):
  train = model_data.iloc[train_idx]
  test = model_data.iloc[test_idx]
  model = cb.CatBoostRegressor(verbose=False, iterations=300)
  model.fit(train.loc[:, feats], train[target])
  _s = pd.Series(model.predict(test[feats]), index=test.index)
  s = pd.concat([s, _s])

model_data['retirement'] = s

<IPython.core.display.Javascript object>

/tmp/ipython-input-1049835522.py:11: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  s = pd.concat([s, _s])
/tmp/ipython-input-1049835522.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data['retirement'] = s


In [ ]:
model_data['z'] = (model_data['retirement'].mul(-1).sub(model_data['retirement'].median())).div(model_data['retirement'].std())
model_data['retirement+'] = (model_data['z'].add(1)).mul(80)

<IPython.core.display.Javascript object>

/tmp/ipython-input-1477168551.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data['z'] = (model_data['retirement'].mul(-1).sub(model_data['retirement'].median())).div(model_data['retirement'].std())
/tmp/ipython-input-1477168551.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data['retirement+'] = (model_data['z'].add(1)).mul(80)


In [ ]:
model_data.head()

<IPython.core.display.Javascript object>

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,pitcher_name,batter,pitcher,events,description,...,spin_axis_adj,release_speed_avg,pfx_x_adj_avg,pfx_z_avg,release_speed_diff,pfx_x_adj_diff,pfx_z_diff,retirement,z,retirement+
0,FC,2025-09-21,96.2,-1.83,5.34,"Doval, Camilo",663624,666808,strikeout,swinging_strike,...,194.0,96.202597,-0.622857,0.307013,-0.002597,0.832857,0.442987,0.000481,0.020748,102.074751
1,FC,2025-09-21,98.2,-1.82,5.24,"Doval, Camilo",663624,666808,NaN,called_strike,...,210.0,96.202597,-0.622857,0.307013,1.997403,0.652857,0.502987,0.006264,0.402612,140.261213
2,SL,2025-09-21,88.4,-1.57,5.43,"Doval, Camilo",663624,666808,NaN,ball,...,73.0,96.202597,-0.622857,0.307013,-7.802597,1.772857,-0.467013,0.002996,0.186807,118.680676
3,FC,2025-09-21,97.9,-1.79,5.32,"Doval, Camilo",663624,666808,NaN,ball,...,222.0,96.202597,-0.622857,0.307013,1.697403,0.752857,0.462987,-0.001915,-0.137519,86.248146
4,FC,2025-09-21,99.1,-1.81,5.28,"Doval, Camilo",663624,666808,NaN,foul,...,210.0,96.202597,-0.622857,0.307013,2.897403,0.182857,0.402987,-0.016875,-1.125427,-12.542719


In [ ]:
agg = model_data.loc[:, ['pitcher','pitcher_name', 'pitch_name', 'retirement+']].groupby(['pitcher','pitcher_name', 'pitch_name'],as_index=False).mean()
_agg = model_data.loc[:, ['pitcher','pitcher_name', 'pitch_name', 'retirement+']].groupby(['pitcher','pitcher_name', 'pitch_name'],as_index=False).count()
agg = agg.merge(_agg.rename(columns={'retirement+':'n'}))
agg.loc[agg['n']>100].sort_values('retirement+',ascending=False).head(20)

<IPython.core.display.Javascript object>

,pitcher,pitcher_name,pitch_name,retirement+,n
244,547973,"Chapman, Aroldis",Slider,214.430621,140
1193,643511,"Rogers, Tyler",Sinker,210.881975,693
1531,661395,"Duran, Jhoan",Split-Finger,202.493467,428
3061,680755,"Fisher, Braydon",Slider,188.671157,300
1351,656730,"Megill, Trevor",Knuckle Curve,186.145962,298
1810,664208,"Maton, Phil",Curveball,184.620264,378
249,548384,"Raley, Brooks",Sweeper,182.261094,138
1194,643511,"Rogers, Tyler",Slider,182.177904,234
5,434378,"Verlander, Justin",Sweeper,180.472107,182
1,434378,"Verlander, Justin",Changeup,180.296923,221


In [ ]:
feats = [
    'release_speed_diff',
    'release_speed'
]

target = 'delta_run_exp'

folds = 3
kf = KFold(n_splits=folds, shuffle=True)
s = pd.Series()
for train_idx, test_idx in kf.split(model_data):
  train = model_data.iloc[train_idx]
  test = model_data.iloc[test_idx]
  model = cb.CatBoostRegressor(verbose=False, iterations=300)
  model.fit(train.loc[:, feats], train[target])
  _s = pd.Series(model.predict(test[feats]), index=test.index)
  s = pd.concat([s, _s])

model_data['yourdad'] = s

<IPython.core.display.Javascript object>

/tmp/ipython-input-3709778471.py:17: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  s = pd.concat([s, _s])
/tmp/ipython-input-3709778471.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data['yourdad'] = s


In [ ]:
model_data['z'] = (model_data['yourdad'].mul(-1).sub(model_data['yourdad'].median())).div(model_data['yourdad'].std())
model_data['yourdad+'] = (model_data['z'].add(1)).mul(80)

<IPython.core.display.Javascript object>

/tmp/ipython-input-3558244084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data['z'] = (model_data['yourdad'].mul(-1).sub(model_data['yourdad'].median())).div(model_data['yourdad'].std())
/tmp/ipython-input-3558244084.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data['yourdad+'] = (model_data['z'].add(1)).mul(80)


In [ ]:
agg = model_data.loc[:, ['pitcher','pitcher_name', 'pitch_name', 'yourdad+']].groupby(['pitcher','pitcher_name', 'pitch_name'],as_index=False).mean()
_agg = model_data.loc[:, ['pitcher','pitcher_name', 'pitch_name', 'yourdad+']].groupby(['pitcher','pitcher_name', 'pitch_name'],as_index=False).count()
agg = agg.merge(_agg.rename(columns={'yourdad+':'n'}))
agg.loc[agg['n']>100].sort_values('yourdad+',ascending=False).head(20)

<IPython.core.display.Javascript object>

,pitcher,pitcher_name,pitch_name,yourdad+,n
4120,695243,"Miller, Mason",Slider,314.769362,433
1193,643511,"Rogers, Tyler",Sinker,280.788818,693
1194,643511,"Rogers, Tyler",Slider,278.207262,234
1531,661395,"Duran, Jhoan",Split-Finger,225.820979,428
1529,661395,"Duran, Jhoan",4-Seam Fastball,206.001531,387
1530,661395,"Duran, Jhoan",Knuckle Curve,182.994457,217
2877,678020,"Halvorsen, Seth",Slider,171.280921,142
1978,666711,"Peguero, Joel",4-Seam Fastball,163.871838,101
4082,694819,"Misiorowski, Jacob",Curveball,163.364454,158
4083,694819,"Misiorowski, Jacob",Slider,161.500410,277


In [ ]:
model_data['pog'] = model_data['retirement'].sub(model_data['yourdad'])
model_data['z'] = (model_data['pog'].mul(-1).sub(model_data['pog'].median())).div(model_data['pog'].std())
model_data['pog+'] = (model_data['z'].add(1)).mul(80)

<IPython.core.display.Javascript object>

/tmp/ipython-input-2305514551.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data['pog'] = model_data['retirement'].sub(model_data['yourdad'])
/tmp/ipython-input-2305514551.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data['z'] = (model_data['pog'].mul(-1).sub(model_data['pog'].median())).div(model_data['pog'].std())
/tmp/ipython-input-2305514551.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [ ]:
agg = model_data.loc[:, ['pitcher','pitcher_name', 'pitch_name', 'pog+']].groupby(['pitcher','pitcher_name', 'pitch_name'],as_index=False).mean()
_agg = model_data.loc[:, ['pitcher','pitcher_name', 'pitch_name', 'pog+']].groupby(['pitcher','pitcher_name', 'pitch_name'],as_index=False).count()
agg = agg.merge(_agg.rename(columns={'pog+':'n'}))
agg.loc[agg['n']>100].sort_values('pog+',ascending=False).head(20)

<IPython.core.display.Javascript object>

,pitcher,pitcher_name,pitch_name,pog+,n
249,548384,"Raley, Brooks",Sweeper,182.989593,138
727,608379,"Wacha, Michael",Changeup,182.567292,694
5,434378,"Verlander, Justin",Sweeper,176.881773,182
244,547973,"Chapman, Aroldis",Slider,174.047523,140
335,573204,"Thielbar, Caleb",Curveball,173.583054,224
1810,664208,"Maton, Phil",Curveball,171.361256,378
176,543056,"Coulombe, Danny",Cutter,171.263078,287
1,434378,"Verlander, Justin",Changeup,171.213285,221
3061,680755,"Fisher, Braydon",Slider,167.984016,300
624,606965,"Devenski, Chris",Changeup,164.246316,109
